# Problema de designação de pessoas **I** para instrumentos **J** e vozes **L** no dia **K**
---

##  i - Conjunto de pessoas
##  J - Conjunto de instrumentos
##  k - Conjunto de dias
##  l - Conjunto de vozes

---

* Mij = Matriz Binária - Pessoa I toca Instrumento J

* Sil = Matriz Binária - Pessoa I toca canta na posição L

* PMj = Peso de cada instrumento J
* PSl = Peso de cada voz L

* Dk = Peso de cada dia K

### Solução
* Xijk - Matriz Binária no R3
* Yilk - Matriz Binária no R3
* Zik -  Matriz binária no R2

* **Se Xijk = 1 => A pessoa "i" tocará o instrumento 'j' no dia 'k'**

* Se Yijl = 1 => A pessoa "i" cantará na posição 'l' no dia 'k'

* **Se Xijk = 0 => A pessoa "i" não tocará o instrumento 'j' no dia 'k'**

* Se Yijl = 0 => A pessoa "i" não cantará na posição 'l' no dia 'k'

* **Somatório em i de Xijk <= 1 -  Um instrumento 'j' só pode ser tocado por uma pessoa 'i' em um dia 'k'**

* Somatório em i de Yilk <= 1 - Uma voz 'l' só pode ser cantada por uma pessoa 'i' em um dia 'k'

* **Somatório em 'j' de Xijk <= 1 - A pessoa "i" tocará somente um instrumento instrumento 'j' no dia 'k'**

* Somatório em 'l' de Yilk <= 1 - A pessoa "i" cantará somente em uma posição "l" no dia "k"

* **Zik - Pessoa i participa no dia k**
* **Zik = 0 se Xijk e yilk = 0**
* **Zik = 1 se Xijk ou yilk = 1**


* **Zik + Zi(k+1) + Zi(k+2) + Zi(k+4) <= 2 para todo i - A pessoa "i" não participará mais do que dois dias seguidos.**


* **Xijk = 0 se PMij = 0**
* Yilk = 0 se PSil = 0


* Maximize Xijk * Mij * PMj * Dk + Yilk * Sil * PSl * Dk

In [42]:
import pandas as pd

file_name = 'Dados Louvor Shalom.xlsx'

sheets_dict = pd.read_excel(file_name, sheet_name=['Instrumento', 'Peso dos Instrumentos', 'Peso dos Dias','Disponibilidade', 'Dias por Mês', 'Vozes', 'Peso das Vozes'])

# Acesse cada sheet pelo nome
#Dias por Mês
P = sheets_dict['Instrumento'].set_index('Pessoas')
V = sheets_dict['Vozes'].set_index('Pessoas')

L = sheets_dict['Peso dos Instrumentos'].set_index('Instrumentos').squeeze()
LV = sheets_dict['Peso das Vozes'].set_index('Voz').squeeze()

D = sheets_dict['Peso dos Dias'].set_index('Dias')

Disp = sheets_dict['Disponibilidade'].set_index('Dias')
DPM = sheets_dict['Dias por Mês'].set_index('Pessoas')

# Convertendo os índices para o tipo datetime
D.index = pd.to_datetime(D.index)
Disp.index = pd.to_datetime(Disp.index)

# Formatando os índices para o formato desejado
D.index = D.index.strftime('%d_%m_%Y')
Disp.index = Disp.index.strftime('%d_%m_%Y')

vetor = pd.Series(1, index=V.columns)
vetor['Nada'] = 0

vetor_inst = pd.Series(1, index=P.columns)
vetor_inst['Nenhum'] = 0

In [43]:
import pandas as pd
from pulp import LpVariable, LpProblem, lpSum, LpMaximize, value

# Definindo a variável X como uma variável de 4 dimensões (i, j, k, l)
X = LpVariable.dicts("X", (P.index, P.columns, D.index, V.columns), cat='Binary')

# Criando o problema de otimização
prob = LpProblem("Problem", LpMaximize)

# Função objetivo atualizada
prob += lpSum([
    X[i][j][k][l] * Disp[i][k] * P.loc[i, j] * L[j] * D['Peso'][k] * V.loc[i, l] * LV[l]
    for i in P.index
    for j in P.columns
    for k in D.index
    for l in V.columns
])

# Um instrumento só é tocado por uma pessoa em cada dia K
for j in P.columns:  # para todo instrumento
    for k in D.index:  # para todo dia
        prob += lpSum([X[i][j][k][l]*vetor_inst[j] for i in P.index for l in V.columns]) <= 1

# Cada pessoa toca somente um instrumento cada dia
for i in P.index:  # cada pessoa
    for k in D.index:  # para todo dia
        prob += lpSum([X[i][j][k][l] for j in P.columns for l in V.columns]) <= 1

# Cada pessoa toca somente o número de dias que se ofereceu por mês
for i in P.index: 
    for k in range(len(D.index) - 3):
        prob += lpSum([
            X[i][j][D.index[k]][l] + X[i][j][D.index[k+1]][l] + X[i][j][D.index[k+2]][l] + X[i][j][D.index[k+3]][l]
            for j in P.columns for l in V.columns
        ]) <= DPM['Quantidade'][i]

# A pessoa não será alocada se não tocar o instrumento específico
for i in P.index:  # pessoas
    for j in P.columns:  # instrumentos
        for k in D.index:  # dias
            for l in V.columns:  # vozes
                if P.loc[i, j] == 0:  # tabela de capacidades. Pessoa i toca o instrumento j.
                    prob += X[i][j][k][l] == 0

# Cada pessoa canta somente uma voz por dia
for i in P.index:  # para cada pessoa
    for k in D.index:  # para cada dia
        prob += lpSum([X[i][j][k][l] for j in P.columns for l in V.columns]) <= 1

# Cada voz é cantada por apenas uma pessoa por dia
for l in V.columns:  # para cada voz
    for k in D.index:  # para cada dia
        prob += lpSum([X[i][j][k][l]*vetor[l] for i in P.index for j in P.columns]) <= 1

# Resolver o problema
prob.solve()

# Tabela de vozes por dia
vozes_por_dia = pd.DataFrame(index=D.index, columns=V.columns)

for k in D.index:
    for l in V.columns:
        for i in P.index:
            for j in P.columns:
                if value(X[i][j][k][l]) == 1:
                    vozes_por_dia.at[k, l] = i

# Tabela de instrumentos por dia
instrumentos_por_dia = pd.DataFrame(index=D.index, columns=P.columns)

for k in D.index:
    for j in P.columns:
        for i in P.index:
            for l in V.columns:
                if value(X[i][j][k][l]) == 1:
                    instrumentos_por_dia.at[k, j] = i

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/henrique/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/eb9019a3ea8d409f8fb88499d2f547b8-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/eb9019a3ea8d409f8fb88499d2f547b8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5447 COLUMNS
At line 75984 RHS
At line 81427 BOUNDS
At line 88838 ENDATA
Problem MODEL has 5442 rows, 7410 columns and 54338 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7525 - 0.01 seconds
Cgl0002I 4615 variables fixed
Cgl0004I processed model has 554 rows, 1378 columns (1378 integer (1378 of which binary)) and 7152 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -7525
Cbc0038I Before mini branch and bound, 1378 integers at

In [44]:
vozes_por_dia.fillna('', inplace=True)
vozes_por_dia.drop(columns=['Nada'])

,Ministração,Voz1,Voz2,Voz3
Dias,,,,
08_06_2024,Ricardo,Marcela,Henrique,Dani
15_06_2024,Rafael,Dani,Marcela,Tassi
22_06_2024,Rafael,Ariane,Henrique,Júlia
29_06_2024,Ricardo,Everlyn,Ariane,Tassi
06_07_2024,Ricardo,Júlia,Marcela,Henrique
13_07_2024,Rafael,Dani,Tassi,Marcela
20_07_2024,Rafael,Ariane,Júlia,Henrique
27_07_2024,Ricardo,Ariane,Everlyn,Tassi
03_08_2024,Ricardo,Henrique,Marcela,Everlyn


In [45]:
instrumentos_por_dia.fillna('', inplace=True)
instrumentos_por_dia.drop(columns=['Nenhum'])

,Bateria,Baixo,Teclado,Violão,Guitarra
Dias,,,,,
08_06_2024,Marcela,Daniel,Henrique,Ricardo,Enzo
15_06_2024,Marcela,Mikael,Mauri,Rafael,Enzo
22_06_2024,Israel,Mikael,Henrique,Rafael,Osvaldo
29_06_2024,Lucas,Gabriel,Vinicius,Ricardo,Daniel
06_07_2024,Marcela,Osvaldo,Henrique,Ricardo,Enzo
13_07_2024,Marcela,Mikael,Mauri,Rafael,Enzo
20_07_2024,Israel,Mikael,Henrique,Rafael,Osvaldo
27_07_2024,Lucas,Gabriel,Bernardo,Ricardo,Daniel
03_08_2024,Marcela,Mikael,Henrique,Ricardo,Daniel
